# Защита персональных данных клиентов
Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

## 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('insurance.csv')

In [2]:
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [3]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Аномальных выбросов, пропусков не наблюдается. Предобрабаывать данные не нужно

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $E$ — единичная матрица

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказания $a$ не изменятся

**Обоснование:** Для доказательства подставим в формулу предсказания произведение матриц $XР$

$$
a' = (XP)w' = XP((XP)^T XP)^{-1} (XP)^T y = XP(P^T X^T XP)^{-1} P^T X^T y = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = 
= XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = XE(X^TX)^{-1}EX^Ty = X(X^T X)^{-1} X^T y = a
$$
**Использовались свойства матриц:**
$$
(XP)^T = P^T X^T, (XP)^{-1} = P^{-1}X^{-1}, PP^{-1} = E
$$

## 3. Алгоритм преобразования

**Алгоритм**

1) Создаем рандомную квадратную матрицу размером в количество признаков.  
2) Проверяем ее на обратимость  
3) Умножаем на нее исходную матрицу признаков. В этот момент момент все данные "теряют" свои значения  
4) Обучаем линейную регрессию на исходной матрице и на "умноженной". Данные точности предсказаний должны быть равны

**Обоснование**

Обоснование возможности кодирования исходной матрицы обратимой изложены в п.2.

## 4. Проверка алгоритма

In [4]:
# Выделение признаков и таргета
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты'] 

In [5]:
# Формирование радомной матрицы для умножения на матрицу признаков
P = np.random.randint(1, 5, size=(4, 4))
print(P)

[[2 1 2 3]
 [3 2 1 3]
 [3 3 1 1]
 [1 4 3 1]]


In [6]:
# Линейная регрессия на исходной матрице признаков
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression() 
model.fit(X, y) 
a = model.predict(X) 
result = r2_score(y, a)
print("R2:", result)

R2: 0.4249455028666801


In [7]:
# Линейная регрессия на произведении матриц
Z = X @ P
model = LinearRegression() 
model.fit(Z, y) 
a = model.predict(Z) 
result = r2_score(y, a)
print("R2:", result)

R2: 0.4249455028666771


## Вывод
Алгоритм реализован. Проведено сравнение качества моделей до и после преобразования и оно одинаково. 